# ⭐ How to install and manage packages and environments. ⭐ 

### Why do you need to care?  

Some code or functions only work on certain versions of Python. So you need to:

- test that your code works on multiple versions
- and/or keep the version that worked when you made your particular project with your particular requirements


# Python versions and packages are files

In [1]:
# the version the interpreter supports
!python --version

Python 3.7.3


- A _python version_ is a version of the Python programming language.
- A python _interpreter_ is an "executable file" that runs the language.
- There's a 1:1 relationship.
- When you install a version of python language, it also installs an interpreter that supports that version

In [4]:
# interpreter
!which python

/Users/lizre/.pyenv/versions/3.7.3/bin/python


The interpreter is at that `path`.


Your python interpreter and packages are just a bunch of files and folders.

Recall that the python _interpreter_ is an "executable file" that runs the language, and it's at this path:

In [16]:
!which python

/Users/lizre/.pyenv/versions/3.7.3/bin/python


`/Users/lizre/.pyenv/versions` contains multiple versions of python/interpreter:
    
<img width="539" alt="image" src="https://user-images.githubusercontent.com/38010821/227318255-14554748-0a27-4db8-911b-c934f8b533fa.png">


Inside each version folder is lib, then python 3.7, 


In [57]:
!cd ~/.pyenv/versions/3.7.3 && ls

bin     etc     images  include lib     share


In [59]:
!cd ~/.pyenv/versions/3.7.3/lib && ls

libpython3.7m.a pkgconfig       python3.7


`lib` has the standard library files that are part of the Python installation, including `site-packages`:

In [71]:
lib = !cd ~/.pyenv/versions/3.7.3/lib/python3.7 && ls
lib[140:150]

['rlcompleter.py',
 'runpy.py',
 'sched.py',
 'secrets.py',
 'selectors.py',
 'shelve.py',
 'shlex.py',
 'shutil.py',
 'signal.py',
 'site-packages']

`site-packages` contains third-party packages:

In [72]:
site_packages = !cd ~/.pyenv/versions/3.7.3/lib/python3.7/site-packages && ls
site_packages[0:5]

['Babel-2.12.1.dist-info',
 'Cython',
 'Cython-0.29.23.dist-info',
 'Flask-2.0.1.dist-info',
 'GitPython-3.1.24.dist-info']

In [36]:
site_packages = !cd ~/.pyenv/versions/3.7.3/lib/python3.7/site-packages && ls

So the packages are _inside_ just ONE version of python.

In [43]:
import pandas as pd 
my_system_python_packages = pd.Series(my_system_python_packages)
my_system_python_packages[200:205]

200    filelock-3.9.0.dist-info
201                      flake8
202      flake8-4.0.1.dist-info
203                       flask
204                      future
dtype: object

In [52]:
my_system_python_packages[384]

'pandas'

When you `pip install`, it installs the package/dependency in the same place as your interpreter. This is your **system environment.**


### Use `pip` to install and manage packages

([but](https://realpython.com/pipenv-guide/) many [people](https://hackernoon.com/reaching-python-development-nirvana-bb5692adf30c) prefer [pipenv](https://medium.com/analytics-vidhya/why-pipenv-over-venv-for-python-projects-a51fb6e4f31e)).


In [13]:
!which pip

/Users/lizre/.pyenv/versions/3.7.3/bin/pip


Starts by searching [PyPI](https://pypi.org/): py package index

In [ ]:
!pip install pandas # Will also get dependencies needed for pandas
!pip list # Should now include pandas

dependencies and dependents:

In [86]:
pandas_deps = !pip show pandas 

In [89]:
pandas_deps

['Name: pandas',
 'Version: 1.3.2',
 'Summary: Powerful data structures for data analysis, time series, and statistics',
 'Home-page: https://pandas.pydata.org',
 'Author: The Pandas Development Team',
 'Author-email: pandas-dev@python.org',
 'License: BSD-3-Clause',
 'Location: /Users/lizre/.pyenv/versions/3.7.3/lib/python3.7/site-packages',
 'Requires: numpy, python-dateutil, pytz',
 'Required-by: cmdstanpy, mlflow, mlxtend, octopy, pandas-profiling, phik, prophet, researchpy, seaborn, sklearn-pandas, statsmodels, visions']

So each Python project on your computer has all their dependencies in the same place.


This is a problem when you have multiple projects that need different package versions. Each project will be looking at this same list of packages. Or if you share code with someone, the code will look in their system environment, which might differ from where the code was developed!



# Solutions

**requirements.txt**

**$PATH**

**virtual environment**: a separate version of this for each project. Each project has a folder with its own Python interpreter (`lib/python`), pip executable, and `site-packages`. 

**not docker:**
<br>venv is only for Python dependencies. Docker is for an entire OS. Use when large-scale and/or dpeloying to a server.

## Manage versions by managing the `$PATH` to executable files

### `PATH` has directories of executable files
- tell our operating system where to look for the executable files associated with commands.
- When you run a `python` command, your OS searches a list of directories to find an executable file with the name `python`. 

The list of directories is in an environment variable called PATH:

In [3]:
!echo $PATH

/Users/lizre/.pyenv/versions/3.7.3/bin:/usr/local/Cellar/pyenv/2.3.15/libexec:/usr/local/Cellar/pyenv/2.3.15/plugins/python-build/bin:/Library/Frameworks/Python.framework/Versions/3.11/bin:/usr/local/Cellar/pyenv-virtualenv/1.2.1/shims:/Users/lizre/.pyenv/shims:/Users/lizre/.pyenv/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/go/bin:/usr/local/share/dotnet:/opt/X11/bin:~/.dotnet/tools


Each directory is separated by `:`.
- /Users/lizre/.pyenv/versions/3.7.3/bin <-- `!which python` returned `/Users/lizre/.pyenv/versions/3.7.3/bin/python`. **so our python executable is stored in this bin!!**
- /Users/lizre/.pyenv/shims:/Users/lizre/.pyenv/bin <-- (shims are like helpers for executables)
- /usr/local/bin
- /usr/bin
- /bin



When you run `ls`, your OS searches a list of directories to find an executable file with the name `ls`:

In [17]:
! cd .. ; ls

HumanEval.jsonl            do-ds
PXL_20230321_022922824.jpg fun-pix
PXL_20230321_023018447.jpg harnesslib
PXL_20230321_023052460.jpg learn-py
PXL_20230321_023107084.jpg secrets.zip


Without `PATH` variable, you'd have to put the full directory of the `ls` command:

In [19]:
! cd .. ; /bin/ls

HumanEval.jsonl            do-ds
PXL_20230321_022922824.jpg fun-pix
PXL_20230321_023018447.jpg harnesslib
PXL_20230321_023052460.jpg learn-py
PXL_20230321_023107084.jpg secrets.zip


In [ ]:
Why more than one directory?
- to have multiple versions of a command installed on your system. 


### The `$PATH` directories are sometimes in `/bin`. 
- `/bin` is not like a garbage bin, it means `binary`. 
- It's a standard directory in mac, for essential executable files 

In [8]:
!ls /bin

[         csh       echo      ksh       mkdir     realpath  stty      wait4path
bash      dash      ed        launchctl mv        rm        sync      zsh
cat       date      expr      link      pax       rmdir     tcsh
chmod     dd        hostname  ln        ps        sh        test
cp        df        kill      ls        pwd       sleep     unlink


in Finder, they look like this:

<img width="489" alt="image" src="https://user-images.githubusercontent.com/38010821/227289808-041439e0-171a-4262-bb3e-28b299a95bb0.png">


The `exec` means they're executable!

The `usr/bin`is for user-installed ones.

In [94]:
bin = !ls /usr/bin
bin[300:315]

['htmltree5.30',
 'ibtool',
 'iconutil',
 'iconv',
 'ictool',
 'id',
 'idlj',
 'imptrace',
 'indent',
 'infocmp',
 'infotocap',
 'install',
 'install_name_tool',
 'instmodsh',
 'instmodsh5.30']

### An _environment variable_ like `PATH` is a variable that's available to a process. 

An environment is not any kind of physical entity--it's just a set of settings (like system time, or user preferences, or language-specific settings like version) and variables.

- like when you set aaa=2 in a jupyter notebook but then you dont have aaa in another notebook or another session. That notebook--session is an _environment_.

You access them with `$`.

In [21]:
!$PATH

/bin/bash: /Users/lizre/.pyenv/versions/3.7.3/bin:/usr/local/Cellar/pyenv/2.3.15/libexec:/usr/local/Cellar/pyenv/2.3.15/plugins/python-build/bin:/Library/Frameworks/Python.framework/Versions/3.11/bin:/usr/local/Cellar/pyenv-virtualenv/1.2.1/shims:/Users/lizre/.pyenv/shims:/Users/lizre/.pyenv/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/go/bin:/usr/local/share/dotnet:/opt/X11/bin:~/.dotnet/tools: No such file or directory


### Set environment variables, like `$PATH`, in zshrc or bashrc _configuration files_

`bash` and `zsh` are `shell`s: command-line interfaces.
- they have differences in syntax and features (eg command completion)

Configure these `shell`s in their "rc" ("run commands") files: `~/.bashrc` and  `~/.zshrc`
- its  called a "run command", not "configuration", because it has commands that are executed by the shell when it starts up
- eg commands that set environment variables
- eg, when `bash` starts up, it runs `~/.bashrc`

In [74]:
!cat ~/.zshrc

export PATH="/opt/homebrew/opt/node@16/bin:$PATH"

`~` means user's home directory (`/Users/lizre`), so identical to: 

In [77]:
!cat /Users/lizre/.zshrc

export PATH="/opt/homebrew/opt/node@16/bin:$PATH"

So when zshrc starts up, it sets `$PATH` to that path.

you can edit it in text editor, or add to it with something like 
<br>`echo 'export PATH="/opt/homebrew/opt/node@16/bin:$PATH"' >> ~/.zshrc`

But that path doesn't have `homebrew`:

In [75]:
! cd /opt && ls

X11


i did find `/usr/local/opt/homebrew/opt/node@16/bin`:, but not the `:$PATH` part:

In [86]:
! cd /usr/local/opt/homebrew/opt/node@16/bin && ls

/bin/bash: line 0: cd: /usr/local/opt/homebrew/opt/node@16/bin: No such file or directory


`/usr` is "Unix System Resources", a standard directory for system software, libraries, and documentation

### pyenv lets you switch between versions of Python by changing your PATH.

[here](https://hackernoon.com/reaching-python-development-nirvana-bb5692adf30c), [here](https://realpython.com/intro-to-pyenv/), [here](https://github.com/pyenv/pyenv).

In [ ]:
https://www.datacamp.com/tutorial/python-developer-set-up

## Make an env

[Beginners should start with venv](https://stackoverflow.com/questions/41573587/what-is-the-difference-between-venv-pyvenv-pyenv-virtualenv-virtualenvwrappe) (but many people prefer [pipenv](https://medium.com/analytics-vidhya/why-pipenv-over-venv-for-python-projects-a51fb6e4f31e)).


In [32]:
!python3 -m venv my_env

`my_env` will contain a copy of whatever python interpreter you used to make it. so make it specifically with `python3`.

In [ ]:
You could also name it .venv.

Avoiding name conflicts: The .venv convention helps to avoid name conflicts with other directories or files that might have the same name as your virtual environment.

Easy to spot: The leading dot in .venv makes it easy to spot and recognize that it is a virtual environment directory.

Ignoring in version control: Many version control systems have a convention of ignoring directories or files that start with a dot, so using .venv makes it easy to ignore your virtual environment directory in version control.
    
    
    
    
    

### Create requirements.txt to record your packages/dependencies

In terminal:

In [8]:
!pipreqs --force

INFO: Successfully saved requirements file in /Users/lizre/Downloads/learn-py/requirements.txt


- You'll see requirements.txt appear in src. It will not exactly match everything you `import` because pipreqs only includes ones not in standard library.
- Can also use `pip freeze > requirements.txt` but it is [harmful because it includes too many things.](https://medium.com/@tomagee/pip-freeze-requirements-txt-considered-harmful-f0bce66cf895); also [pypi says not to use pip freeze](https://pypi.org/project/pipreqs/).



In [10]:
!cat requirements.txt

pandas==1.3.2


To use:

In [12]:
!pip install -r requirements.txt

In [15]:
!cd my_env && ls

bin        include    lib        pyvenv.cfg


Creates a folder called `my_env`, with Python, pip and `site-packages`:

In [ ]:
├── bin
│   ├── activate
│   ├── activate.csh
│   ├── activate.fish
│   ├── easy_install
│   ├── pip
│   ├── pip3
│   ├── python 
│   └── python3.7
├── include
├── lib
│   └── python3.7
│       └── site-packages
└── pyvenv.cfg

lib: a copy of Python.

## Enter/activate env

`activate` scripts tell your shell to use the venv’s Python executable and its `site-packages`, instead of the system ones.

so just run `activate`:

In [19]:
!source my_env/bin/activate 

### What activating does

Now prompt has env name:

![image](https://user-images.githubusercontent.com/38010821/153217272-690c3c2d-7035-474b-88ee-3ba1238a2d21.png)


And now, instead of looking for Python in `/Users/lizre/.pyenv/versions/3.7.3/bin/python`, it's looking in `my_env`:

In [92]:
!source my_env/bin/activate && which python

/Users/lizre/Downloads/learn-py/my_env/bin/python


And `my_env` is at the beginning of PATH, meaning the venv is the first directory used:

In [98]:
!source my_env/bin/activate && echo $PATH

/Users/lizre/Downloads/learn-py/my_env/bin:/Users/lizre/.pyenv/versions/3.7.3/bin:/usr/local/Cellar/pyenv/2.0.6/libexec:/usr/local/Cellar/pyenv/2.0.6/plugins/python-build/bin:/Users/lizre/.pyenv/shims:/Users/lizre/.pyenv/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin


Whereas outside the venv, `my_env` is not in PATH at all:

In [101]:
!echo $PATH

/Users/lizre/.pyenv/versions/3.7.3/bin:/usr/local/Cellar/pyenv/2.0.6/libexec:/usr/local/Cellar/pyenv/2.0.6/plugins/python-build/bin:/Users/lizre/.pyenv/shims:/Users/lizre/.pyenv/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin


### Install packages in the venv

`lib` &  `site-packages`: holds dependencies/packages you install in the venv:

In [34]:
!cd my_env/lib/python3.7/site-packages && ls

__pycache__                 pkg_resources
easy_install.py             setuptools
pip                         setuptools-40.8.0.dist-info
pip-19.0.3.dist-info


In [35]:
!source my_env/bin/activate && pip install numpy

  Using cached https://files.pythonhosted.org/packages/09/8c/ae037b8643aaa405b666c167f48550c1ce6b7c589fe5540de6d83e5931ca/numpy-1.21.5-cp37-cp37m-macosx_10_9_x86_64.whl
You are using pip version 19.0.3, however version 22.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Now numpy's there:

In [36]:
!cd my_env/lib/python3.7/site-packages && ls

__pycache__                 pip-19.0.3.dist-info
easy_install.py             pkg_resources
numpy                       setuptools
numpy-1.21.5.dist-info      setuptools-40.8.0.dist-info
pip


## Exit env

In [ ]:
!deactivate

Because a venv is just a folder, to delete one, just delete its folder.

# TODO
### [envs inside jupyter](https://stephen-odaibo.medium.com/docker-containers-python-virtual-environments-virtual-machines-d00aa9b8475)
- .env doesn't impact your current bash environment, just your python environment after calling load_dotenv()



### in codespaces
https://code.visualstudio.com/docs/python/environments

### [use cookiecutter](https://www.datacamp.com/community/tutorials/python-developer-set-up) for scaffolding/templating